In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Endometrioid_Cancer"
cohort = "GSE73551"

# Input paths
in_trait_dir = "../../input/GEO/Endometrioid_Cancer"
in_cohort_dir = "../../input/GEO/Endometrioid_Cancer/GSE73551"

# Output paths
out_data_file = "../../output/preprocess/Endometrioid_Cancer/GSE73551.csv"
out_gene_data_file = "../../output/preprocess/Endometrioid_Cancer/gene_data/GSE73551.csv"
out_clinical_data_file = "../../output/preprocess/Endometrioid_Cancer/clinical_data/GSE73551.csv"
json_path = "../../output/preprocess/Endometrioid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Prior Knowledge Transfer Across Transcriptional Datasets Using Compositional Statistics [Tumor]"
!Series_summary	"An expert-pathologist-reviewed epithelial ovarian cancer reference library (n = 50) used to assign the histopathology of epithelial ovarian cell lines using compositional statistics and random gene-sets"
!Series_overall_design	"In the study presented here, we applied Gene Expression Compositional Assignment (GECA) to epithelial ovarian cell lines (GSE73637), using first a reference library of solid tumors (expO [http://www.intgen.org/expo/]) and then a second library of expert pathologically-reviewed epithelial ovarian cancer samples"
Sample Characteristics Dictionary:
{0: ['cell type: MUCINOUS', 'cell type: CLEAR CELL', 'cell type: SEROUS LOW-GRADE', 'cell type: ENDOMETRIOID', 'cell type: SEROUS'], 1: ['tissue: Epithelial Ovarian Cancer']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series title and overall design, this dataset appears to contain transcriptional data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Endometrioid_Cancer), we can use the cell type information in row 0
trait_row = 0

# Age data is not available in the sample characteristics
age_row = None

# Gender data is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert the cell type value to a binary trait for Endometrioid Cancer
    1 for endometrioid, 0 for other types
    """
    if pd.isna(value):
        return None
        
    # Extract the value after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if the cell type is endometrioid (case insensitive)
    if 'endometrioid' in value.lower():
        return 1
    else:
        return 0

# Age conversion function (not used since age data is not available)
def convert_age(value):
    return None

# Gender conversion function (not used since gender data is not available)
def convert_gender(value):
    return None

# 3. Save Metadata
# Determine if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Since we don't have a CSV file, we'll use the sample characteristics dictionary directly
    # Create the clinical_data dataframe from the sample characteristics dictionary
    sample_char_dict = {0: ['cell type: MUCINOUS', 'cell type: CLEAR CELL', 'cell type: SEROUS LOW-GRADE', 
                           'cell type: ENDOMETRIOID', 'cell type: SEROUS'], 
                        1: ['tissue: Epithelial Ovarian Cancer']}
    
    # Convert sample characteristics dictionary to a dataframe
    # We need to create a dataframe with samples as columns and characteristics as rows
    # First, extract unique values for each characteristic
    unique_values = {}
    for key, values in sample_char_dict.items():
        unique_values[key] = values
    
    # Create a dummy dataframe with this information
    # In a real dataset, we would have sample IDs as columns, but here we'll use the cell types as samples
    samples = []
    for value in unique_values[0]:  # Use cell types as samples
        sample_name = value.split(": ")[1] if ": " in value else value
        samples.append(sample_name)
    
    # Create the dataframe
    data = {sample: [] for sample in samples}
    for i, cell_type in enumerate(samples):
        data[cell_type] = [f"cell type: {cell_type}" if j == 0 else unique_values[j][0] for j in range(len(unique_values))]
    
    clinical_data = pd.DataFrame(data)
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(clinical_features)
    print("Preview of clinical features:")
    print(preview)
    
    # Save the clinical features to the specified file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features:
{'MUCINOUS': [0.0], 'CLEAR CELL': [0.0], 'SEROUS LOW-GRADE': [0.0], 'ENDOMETRIOID': [1.0], 'SEROUS': [0.0]}
Clinical features saved to ../../output/preprocess/Endometrioid_Cancer/clinical_data/GSE73551.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 59
Header line: "ID_REF"	"GSM1897741"	"GSM1897744"	"GSM1897746"	"GSM1897748"	"GSM1897750"	"GSM1897752"	"GSM1897753"	"GSM1897755"	"GSM1897757"	"GSM1897759"	"GSM1897761"	"GSM1897763"	"GSM1897765"	"GSM1897767"	"GSM1897769"	"GSM1897770"	"GSM1897772"	"GSM1897774"	"GSM1897776"	"GSM1897778"	"GSM1897780"	"GSM1897782"	"GSM1897784"	"GSM1897786"	"GSM1897787"	"GSM1897789"	"GSM1897792"	"GSM1897794"	"GSM1897795"	"GSM1897797"	"GSM1897799"	"GSM1897801"	"GSM1897802"	"GSM1897804"	"GSM1897806"	"GSM1897808"	"GSM1897810"	"GSM1897812"	"GSM1897814"	"GSM1897816"	"GSM1897818"	"GSM1897820"	"GSM1897822"	"GSM1897823"	"GSM1897825"	"GSM1897827"	"GSM1897829"	"GSM1897831"	"GSM1897833"	"GSM1897835"
First data line: 1	5.403790297	4.892553637	5.326199224	5.170940973	5.427630826	5.516932949	5.470710937	4.98660365	5.133514638	5.230458349	4.990772814	5.662734397	5.696864862	5.147933271	4.86309129	5.394637232	5.381253066	5.842655059	5.545258599	5.707996433	5.684696894	5.428043787	5.099403081	5.5725

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers shown in the data are numeric IDs (1, 2, 3, etc.), not human gene symbols.
# These are likely probe IDs or some other form of identifier that needs to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE73551
Line 6: !Series_title = Prior Knowledge Transfer Across Transcriptional Datasets Using Compositional Statistics [Tumor]
Line 7: !Series_geo_accession = GSE73551
Line 8: !Series_status = Public on Nov 08 2016
Line 9: !Series_submission_date = Sep 29 2015
Line 10: !Series_last_update_date = Nov 10 2016
Line 11: !Series_pubmed_id = 27353327
Line 12: !Series_summary = An expert-pathologist-reviewed epithelial ovarian cancer reference library (n = 50) used to assign the histopathology of epithelial ovarian cell lines using compositional statistics and random gene-sets
Line 13: !Series_overall_design = In the study presented here, we applied Gene Expression Compositional Assignment (GECA) to epithelial o


Gene annotation preview:
{'ID': [1, 2, 3, 4, 5], 'ProbeSetID': ['200000_s_at', '200001_at', '200002_at', '200003_s_at', '200004_at'], 'GeneSymbol': ['PRPF8', 'CAPNS1', 'RPL35', 'RPL28', 'EIF4G2'], 'Array': ['Ovarian Cancer DSA', 'Ovarian Cancer DSA', 'Ovarian Cancer DSA', 'Ovarian Cancer DSA', 'Ovarian Cancer DSA'], 'Annotation Date': ['10-Jan-11', '10-Jan-11', '10-Jan-11', '10-Jan-11', '10-Jan-11'], 'Sequence Type': ['Affymetrix human normalisation control', 'Affymetrix human normalisation control', 'Affymetrix human normalisation control', 'Affymetrix human normalisation control', 'Affymetrix human normalisation control'], 'Ensembl Version': ['release 60', 'release 60', 'release 60', 'release 60', 'release 60'], 'Ensembl Genome Version': ['GRCh37', 'GRCh37', 'GRCh37', 'GRCh37', 'GRCh37'], 'Orientation / Description': ['Sense (Fully Exonic)', 'Sense (Fully Exonic)', 'Sense (Fully Exonic)', 'Sense (Fully Exonic)', 'Sense (Fully Exonic)'], 'No. probes aligned': ['9', '8', '8', '9', '11

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the appropriate columns in the gene annotation data
# From the preview, we can see that 'ID' in gene_annotation corresponds to the numeric IDs (1, 2, 3) in gene_data
# The 'GeneSymbol' column contains the human gene symbols we want to map to

# 2. Create a gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col="ID", gene_col="GeneSymbol")
print("Gene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression
# This will handle the many-to-many relation between probes and genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"\nGene data shape after mapping: {gene_data.shape}")
print("Gene data preview (first 5 genes, first 3 samples):")
if gene_data.shape[0] > 0 and gene_data.shape[1] > 0:
    preview_cols = min(3, gene_data.shape[1])
    print(gene_data.iloc[:5, :preview_cols])

# Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Gene mapping preview (first 5 rows):
  ID    Gene
0  1   PRPF8
1  2  CAPNS1
2  3   RPL35
3  4   RPL28
4  5  EIF4G2



Gene data shape after mapping: (20172, 50)
Gene data preview (first 5 genes, first 3 samples):
       GSM1897741  GSM1897744  GSM1897746
Gene                                     
A1BG     5.331976    5.578752    5.536849
A1CF     3.920673    2.936927    3.194641
A2LD1   14.586115   16.210582   14.578840
A2M     14.252609   10.769140   10.335943
A2ML1    3.600177    2.762048    2.728659



Gene expression data saved to ../../output/preprocess/Endometrioid_Cancer/gene_data/GSE73551.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note="Dataset contains gene expression data but lacks clear trait indicators for Duchenne Muscular Dystrophy status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs healthy samples."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (20036, 50)
First few genes with their expression values after normalization:
         GSM1897741  GSM1897744  GSM1897746  GSM1897748  GSM1897750  \
Gene                                                                  
A1BG       5.331976    5.578752    5.536849    5.161539    5.352578   
A1CF       3.920673    2.936927    3.194641    3.082743    2.935914   
A2M       14.252609   10.769140   10.335943   10.783329   11.617907   
A2ML1      3.600177    2.762048    2.728659    3.291211    3.406832   
A3GALT2    6.455418    6.153991    6.062100    5.696409    5.895500   

         GSM1897752  GSM1897753  GSM1897755  GSM1897757  GSM1897759  ...  \
Gene                                                                 ...   
A1BG       5.062424    5.628661    4.694100    5.454426    5.214719  ...   
A1CF       3.430515    6.307725    3.109970    3.176425    3.403035  ...   
A2M        7.886267   10.788412   11.061511   11.618258   10.487040  ...   
A2ML1      6.553

Normalized gene data saved to ../../output/preprocess/Endometrioid_Cancer/gene_data/GSE73551.csv


Raw clinical data shape: (2, 51)
Clinical features:
                     GSM1897741  GSM1897744  GSM1897746  GSM1897748  \
Endometrioid_Cancer         0.0         0.0         0.0         1.0   

                     GSM1897750  GSM1897752  GSM1897753  GSM1897755  \
Endometrioid_Cancer         1.0         0.0         0.0         0.0   

                     GSM1897757  GSM1897759  ...  GSM1897818  GSM1897820  \
Endometrioid_Cancer         0.0         0.0  ...         0.0         0.0   

                     GSM1897822  GSM1897823  GSM1897825  GSM1897827  \
Endometrioid_Cancer         0.0         0.0         0.0         0.0   

                     GSM1897829  GSM1897831  GSM1897833  GSM1897835  
Endometrioid_Cancer         0.0         0.0         0.0         0.0  

[1 rows x 50 columns]
Clinical features saved to ../../output/preprocess/Endometrioid_Cancer/clinical_data/GSE73551.csv
Linked data shape: (50, 20037)
Linked data preview (first 5 rows, first 5 columns):
            Endometri

Data shape after handling missing values: (50, 20037)
For the feature 'Endometrioid_Cancer', the least common label is '1.0' with 9 occurrences. This represents 18.00% of the dataset.
The distribution of the feature 'Endometrioid_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Endometrioid_Cancer/GSE73551.csv
